In [ ]:
import pandas as pd
import zipfile
import requests
from io import BytesIO
import os
from scipy.spatial import KDTree
import numpy as np

Charger les données de DVF

In [5]:
#les données suivantes sont les données dans herault  (34)
url0 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2024/departements/34.csv.gz" # 2024 (seulement Q1)
url1 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2023/departements/34.csv.gz" # 2023
url2 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2022/departements/34.csv.gz" # 2022
url3 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2021/departements/34.csv.gz" # 2021
url4 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2020/departements/34.csv.gz" # 2020
url5 = "https://files.data.gouv.fr/geo-dvf/latest/csv/2019/departements/34.csv.gz" # 2019

In [6]:
# Load data from URLs
def load_data(url):
    
    df = pd.read_csv(url,compression="gzip", low_memory=False)  
    df.dropna(axis=1, how='all', inplace=True)
    df = df.dropna(subset=['code_postal']) # enlever les valeurs nulles de code postal
    df['code_postal'] = df['code_postal'].astype(int).astype(str)
    #code_mask = df['code_postal'].isin(codes_postaux_metropole) # filtre code postal pour Montpellier Méditerranée Métropole
    #df =df[code_mask]
    df['date_mutation']= pd.to_datetime(df['date_mutation'], format='%Y-%m-%d', errors='coerce')
    df['year'] = df['date_mutation'].dt.year
    return df
    
df0 = load_data(url0)
df1 = load_data(url1)
df2 = load_data(url2)
df3 = load_data(url3)
df4 = load_data(url4)
df5 = load_data(url5)

HTTPError: HTTP Error 404: Not Found

In [ ]:
df0.head(2)

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,year
0,2024-179483,2024-01-03,1,Vente,500000.0,21.0,NaN,RUE DE LA BERGERIE DE CAUNELLE,0009,34990,...,127.0,4.0,S,sols,NaN,NaN,638.0,3.800856,43.616081,2024
1,2024-179483,2024-01-03,1,Vente,500000.0,21.0,NaN,RUE DE LA BERGERIE DE CAUNELLE,0009,34990,...,NaN,0.0,S,sols,NaN,NaN,638.0,3.800856,43.616081,2024


In [ ]:
df0.shape[0], df1.shape[0], df2.shape[0], df3.shape[0], df4.shape[0], df5.shape[0]  

(30085, 77463, 94972, 95505, 73220, 39385)

Combine les 5 fichiers

In [ ]:
df_combined = pd.concat([df0, df1, df2, df3, df4, df5], ignore_index=True)

In [ ]:
df_combined.shape[0]

410630

In [ ]:
select_columns = ['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'numero_volume', 'lot1_surface_carrez',
       'lot2_surface_carrez', 'lot3_surface_carrez', 'lot4_surface_carrez',
       'lot5_surface_carrez', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales']
df_s = df_combined[select_columns]

In [ ]:
df12= df_s[df_s['lot2_surface_carrez'].notnull()]  #| df['latitude'].isnull()]
df12.head(2)

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,numero_volume,lot1_surface_carrez,lot2_surface_carrez,lot3_surface_carrez,lot4_surface_carrez,lot5_surface_carrez,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales
30,2024-179499,2024-01-04,1,Vente,118000.0,0241,34400,34145,Lunel,34,NaN,NaN,51.58,NaN,NaN,NaN,2.0,Appartement,52.0,2.0
31,2024-179499,2024-01-04,1,Vente,118000.0,0241,34400,34145,Lunel,34,NaN,NaN,51.58,NaN,NaN,NaN,3.0,Dépendance,NaN,0.0


In [ ]:
df_combined.columns

Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'id_parcelle', 'numero_volume',
       'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude',
       'year', 'ancien_code_commune', 'ancien_nom_commune',
       'ancien_id_parcelle'],
      dtype='object')

In [ ]:
# Reserve les colonnes suivantes
columns_to_keep = [
    'valeur_fonciere',               
    'type_local',                   
    'surface_reelle_bati',           
    'surface_terrain',               
    'nombre_pieces_principales',     
    'nombre_lots',                  
    'code_postal',                  
    'nom_commune',                   
    'code_commune',                  
    'code_departement',              
    'longitude',                     
    'latitude',                      
    'date_mutation',
    'nature_mutation',                 
    'year',                        
]

# filtre les colonnes
df_filtered = df_combined[columns_to_keep]

In [ ]:
df_filtered.head(2)

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,code_departement,longitude,latitude,date_mutation,nature_mutation,year
0,500000.0,Maison,127.0,638.0,4.0,0,34990,Juvignac,34123,34,3.800856,43.616081,2024-01-03,Vente,2024
1,500000.0,Dépendance,NaN,638.0,0.0,0,34990,Juvignac,34123,34,3.800856,43.616081,2024-01-03,Vente,2024


In [ ]:
df_grouped = df_filtered.groupby('type_local').size()
df_grouped

type_local
Appartement                                  83245
Dépendance                                  110082
Local industriel. commercial ou assimilé     13726
Maison                                       63485
dtype: int64

In [ ]:
df_type_local = df_filtered[df_filtered['type_local'] .isin(['Appartement','Maison'])]
df_type_local.head()

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,code_departement,longitude,latitude,date_mutation,nature_mutation,year
0,500000.0,Maison,127.0,638.0,4.0,0,34990,Juvignac,34123,34,3.800856,43.616081,2024-01-03,Vente,2024
3,40000.0,Appartement,17.0,NaN,1.0,1,34080,Montpellier,34172,34,3.837947,43.613467,2024-01-05,Vente,2024
8,263660.0,Maison,65.0,NaN,3.0,1,34090,Montpellier,34172,34,3.852466,43.638047,2024-01-05,Vente,2024
11,160500.0,Appartement,43.0,NaN,2.0,2,34090,Montpellier,34172,34,3.868303,43.636036,2024-01-05,Vente,2024
13,250000.0,Appartement,66.0,NaN,3.0,2,34570,Pignan,34202,34,3.770587,43.586034,2024-01-08,Vente,2024


Dépendance : Garage / Cave / Grenier / Abri de Jardin / Piscine / Atelier 
Local industriel, commercial ou assimilé : Local industriel / Local commercial / Asimilé  

In [ ]:
df_grouped = df_filtered.groupby('nature_mutation').size()
df_grouped

nature_mutation
Adjudication                            1203
Echange                                 2593
Expropriation                             13
Vente                                 369219
Vente en l'état futur d'achèvement     35971
Vente terrain à bâtir                   1631
dtype: int64

In [ ]:
print(df_type_local.columns)


Index(['valeur_fonciere', 'type_local', 'surface_reelle_bati',
       'surface_terrain', 'nombre_pieces_principales', 'nombre_lots',
       'code_postal', 'nom_commune', 'code_commune', 'code_departement',
       'longitude', 'latitude', 'date_mutation', 'nature_mutation', 'year'],
      dtype='object')


On va garder : Vente /  Vente en l'état futur d'achèvement /Vente terrain à bâtir   

In [ ]:
df_vente = df_type_local[df_type_local['nature_mutation'].isin([
    'Vente', 
    'Vente en l\'état futur d\'achèvement', 
    'Vente terrain à bâtir'
])]
df_vente.head()

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,code_departement,longitude,latitude,date_mutation,nature_mutation,year
0,500000.0,Maison,127.0,638.0,4.0,0,34990,Juvignac,34123,34,3.800856,43.616081,2024-01-03,Vente,2024
3,40000.0,Appartement,17.0,NaN,1.0,1,34080,Montpellier,34172,34,3.837947,43.613467,2024-01-05,Vente,2024
8,263660.0,Maison,65.0,NaN,3.0,1,34090,Montpellier,34172,34,3.852466,43.638047,2024-01-05,Vente,2024
11,160500.0,Appartement,43.0,NaN,2.0,2,34090,Montpellier,34172,34,3.868303,43.636036,2024-01-05,Vente,2024
13,250000.0,Appartement,66.0,NaN,3.0,2,34570,Pignan,34202,34,3.770587,43.586034,2024-01-08,Vente,2024


In [ ]:
#vérifie les valeurs nulles dans chaque colonne
missing_values_per_column = df_vente.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere                 53
type_local                       0
surface_reelle_bati              2
surface_terrain              74652
nombre_pieces_principales        2
nombre_lots                      0
code_postal                      0
nom_commune                      0
code_commune                     0
code_departement                 0
longitude                     1771
latitude                      1771
date_mutation                    0
nature_mutation                  0
year                             0
dtype: int64


In [ ]:
# total des valeurs manquantes
total_missing_values = df_vente.isnull().sum().sum()
print(f"Total missing values: {total_missing_values}")

Total missing values: 78251


In [ ]:
df_longitude_latitude = df_vente[df_vente['longitude'].isnull()|df_vente['latitude'].isnull()]

df_longitude_latitude

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,code_departement,longitude,latitude,date_mutation,nature_mutation,year
2097,6500.0,Maison,45.0,NaN,3.0,0,34700,Les Plans,34205,34,NaN,NaN,2024-01-29,Vente,2024
3706,560000.0,Maison,43.0,195.0,2.0,0,34000,Montpellier,34172,34,NaN,NaN,2024-02-20,Vente,2024
5000,8000.0,Maison,178.0,1152.0,6.0,0,34660,Cournonsec,34087,34,NaN,NaN,2024-02-29,Vente,2024
5003,8000.0,Maison,178.0,1200.0,6.0,0,34660,Cournonsec,34087,34,NaN,NaN,2024-02-29,Vente,2024
7477,400000.0,Maison,107.0,1133.0,3.0,0,34660,Cournonsec,34087,34,NaN,NaN,2024-03-14,Vente,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409228,180000.0,Appartement,34.0,NaN,2.0,1,34300,Agde,34003,34,NaN,NaN,2019-12-06,Vente,2019
409402,60000.0,Appartement,34.0,NaN,2.0,1,34300,Agde,34003,34,NaN,NaN,2019-11-29,Vente,2019
409710,530000.0,Maison,144.0,878.0,5.0,0,34120,Castelnau-de-Guers,34056,34,NaN,NaN,2019-12-06,Vente,2019
410049,103000.0,Appartement,32.0,NaN,2.0,1,34340,Marseillan,34150,34,NaN,NaN,2019-12-16,Vente,2019


In [ ]:
df_vente.dtypes

valeur_fonciere                     float64
type_local                           object
surface_reelle_bati                 float64
surface_terrain                     float64
nombre_pieces_principales           float64
nombre_lots                           int64
code_postal                          object
nom_commune                          object
code_commune                          int64
code_departement                      int64
longitude                           float64
latitude                            float64
date_mutation                datetime64[ns]
nature_mutation                      object
year                                  int64
dtype: object

In [ ]:
df_popu = pd.read_csv('herault_population_2019_2024.csv')
df_popu.head()

,code_commune,year,population
0,34091,2024,56
1,34091,2021,54
2,34092,2024,967
3,34092,2023,977
4,34093,2025,147


In [ ]:
df_vente['code_postal'] = df_vente['code_postal'].astype(int)

C:\Users\yutin\AppData\Local\Temp\ipykernel_35944\740799383.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vente['code_postal'] = df_vente['code_postal'].astype(int)


In [ ]:
#merge df_popu + de_vente
df_merged = pd.merge(df_vente, df_popu, on=['code_commune', 'year'], how='inner')

In [ ]:
df_merged = df_merged.drop(columns=['code_departement', 'nature_mutation'])

In [ ]:
df_merged['nombre_pieces_principales'] = df_merged['nombre_pieces_principales'].fillna(0)

In [ ]:
df_merged['nombre_pieces_principales'] = df_merged['nombre_pieces_principales'].astype(int)

In [ ]:
df_merged.head(2)

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,date_mutation,year,population
0,500000.0,Maison,127.0,638.0,4,0,34990,Juvignac,34123,3.800856,43.616081,2024-01-03,2024,11344
1,144000.0,Appartement,47.0,NaN,2,2,34990,Juvignac,34123,3.809723,43.618576,2024-01-12,2024,11344


In [ ]:
df_rev = pd.read_csv('herault_revenu.csv')
df_rev.head()

,code_commune,rev_fisc_moyen_reference
0,34002,21.558788
1,34004,17.478192
2,34006,19.962507
3,34007,17.640876
4,34011,22.605179


In [ ]:
#merge df_rev + de_merged
df_merged_rev = pd.merge(df_merged, df_rev, on=['code_commune'], how='inner')

In [ ]:
df_merged_rev.head(2)

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,date_mutation,year,population,rev_fisc_moyen_reference
0,500000.0,Maison,127.0,638.0,4,0,34990,Juvignac,34123,3.800856,43.616081,2024-01-03,2024,11344,28.720774
1,144000.0,Appartement,47.0,NaN,2,2,34990,Juvignac,34123,3.809723,43.618576,2024-01-12,2024,11344,28.720774


In [ ]:
df_merged_rev['rev_fisc_moyen_reference'] = df_merged_rev['rev_fisc_moyen_reference'].round(2)

In [ ]:
df_merged_rev.shape[0]

129940

In [ ]:
missing_values_per_column = df_merged_rev.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere                 52
type_local                       0
surface_reelle_bati              1
surface_terrain              66497
nombre_pieces_principales        0
nombre_lots                      0
code_postal                      0
nom_commune                      0
code_commune                     0
longitude                     1414
latitude                      1414
date_mutation                    0
year                             0
population                       0
rev_fisc_moyen_reference        10
dtype: int64


In [ ]:
# Delete rows where 'valeur_fonciere' is null
df_merged_rev = df_merged_rev.dropna(subset=['valeur_fonciere'])

In [ ]:
df_merged_rev['surface_terrain'] = df_merged_rev['surface_terrain'].fillna(0)

In [ ]:
df_merged_rev.head(2)

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,date_mutation,year,population,rev_fisc_moyen_reference
0,500000.0,Maison,127.0,638.0,4,0,34990,Juvignac,34123,3.800856,43.616081,2024-01-03,2024,11344,28.72
1,144000.0,Appartement,47.0,0.0,2,2,34990,Juvignac,34123,3.809723,43.618576,2024-01-12,2024,11344,28.72


In [ ]:
df_merged_rev = df_merged_rev.dropna(subset=['surface_reelle_bati'])

In [ ]:
df_merged_rev['surface_reelle_bati'] = df_merged_rev['surface_reelle_bati'].astype(int)

In [ ]:
df_merged_rev['valeur_fonciere'] = df_merged_rev['valeur_fonciere'].astype(int)

In [ ]:
df_merged_rev['surface_terrain'] = df_merged_rev['surface_terrain'].astype(int)

In [ ]:
df_merged_rev.head(2)

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,date_mutation,year,population,rev_fisc_moyen_reference
0,500000,Maison,127,638,4,0,34990,Juvignac,34123,3.800856,43.616081,2024-01-03,2024,11344,28.72
1,144000,Appartement,47,0,2,2,34990,Juvignac,34123,3.809723,43.618576,2024-01-12,2024,11344,28.72


In [ ]:
df_merged_rev = df_merged_rev.drop(columns=['date_mutation'])

In [ ]:
missing_values_per_column = df_merged_rev.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere                 0
type_local                      0
surface_reelle_bati             0
surface_terrain                 0
nombre_pieces_principales       0
nombre_lots                     0
code_postal                     0
nom_commune                     0
code_commune                    0
longitude                    1413
latitude                     1413
year                            0
population                      0
rev_fisc_moyen_reference       10
dtype: int64


In [ ]:
df_merged_rev.head()

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,year,population,rev_fisc_moyen_reference
0,500000,Maison,127,638,4,0,34990,Juvignac,34123,3.800856,43.616081,2024,11344,28.72
1,144000,Appartement,47,0,2,2,34990,Juvignac,34123,3.809723,43.618576,2024,11344,28.72
2,183000,Appartement,68,0,3,2,34990,Juvignac,34123,3.811186,43.617937,2024,11344,28.72
3,255350,Appartement,78,0,3,2,34990,Juvignac,34123,3.803852,43.613413,2024,11344,28.72
4,174000,Appartement,59,0,3,1,34990,Juvignac,34123,3.801290,43.613778,2024,11344,28.72


In [ ]:
#imputation longitud et latitude par la moyenne 

# Group by 'code_commune' and fill missing 'longitude' and 'latitude' with the group's mean
df_merged_rev['longitude'] = df_merged_rev.groupby('code_commune')['longitude'].transform(lambda x: x.fillna(x.mean()))
df_merged_rev['latitude'] = df_merged_rev.groupby('code_commune')['latitude'].transform(lambda x: x.fillna(x.mean()))


In [ ]:
missing_values_per_column = df_merged_rev.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere               0
type_local                    0
surface_reelle_bati           0
surface_terrain               0
nombre_pieces_principales     0
nombre_lots                   0
code_postal                   0
nom_commune                   0
code_commune                  0
longitude                     0
latitude                      0
year                          0
population                    0
rev_fisc_moyen_reference     10
dtype: int64


Il reste 10 valeur nulles dans rev_fisc_moyen_reference.

In [ ]:
# calcule le moyen de 'rev_fisc_moyen_reference
mean_value = df_merged_rev['rev_fisc_moyen_reference'].mean()

# fillna le moyen de 'rev_fisc_moyen_reference
df_merged_rev['rev_fisc_moyen_reference'] = df_merged_rev['rev_fisc_moyen_reference'].fillna(mean_value)


In [ ]:
missing_values_per_column = df_merged_rev.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere              0
type_local                   0
surface_reelle_bati          0
surface_terrain              0
nombre_pieces_principales    0
nombre_lots                  0
code_postal                  0
nom_commune                  0
code_commune                 0
longitude                    0
latitude                     0
year                         0
population                   0
rev_fisc_moyen_reference     0
dtype: int64


Il n'y a plus de valeur nulles. 

In [ ]:
df_cho = pd.read_csv('taux_chomage.csv')
df_cho.head()

,code_commune,unemployment_rate
0,34001,25.75
1,34002,27.99
2,34003,39.01
3,34004,4.55
4,34005,51.96


In [ ]:
#merge df_merged_rev + dedf_cho
df_merged_cho = pd.merge(df_merged_rev, df_cho, on=['code_commune'], how='inner')

In [ ]:
df_merged_cho.head(2)

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,year,population,rev_fisc_moyen_reference,unemployment_rate
0,500000,Maison,127,638,4,0,34990,Juvignac,34123,3.800856,43.616081,2024,11344,28.72,28.32
1,144000,Appartement,47,0,2,2,34990,Juvignac,34123,3.809723,43.618576,2024,11344,28.72,28.32


In [ ]:
missing_values_per_column = df_merged_cho.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere              0
type_local                   0
surface_reelle_bati          0
surface_terrain              0
nombre_pieces_principales    0
nombre_lots                  0
code_postal                  0
nom_commune                  0
code_commune                 0
longitude                    0
latitude                     0
year                         0
population                   0
rev_fisc_moyen_reference     0
unemployment_rate            1
dtype: int64


In [ ]:
valeurnulles = df_merged_cho[df_merged_cho['unemployment_rate'].isnull()]
valeurnulles

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,year,population,rev_fisc_moyen_reference,unemployment_rate
129881,452860,Maison,105,81,3,0,34520,Saint-Félix-de-l'Héras,34253,3.290564,43.836364,2021,35,24.04,NaN


In [ ]:
# il ny a pas de chomeur dans cette commune 34253, donc on met 0  
df_merged_cho['unemployment_rate'] = df_merged_cho['unemployment_rate'].fillna(0)


In [ ]:
missing_values_per_column = df_merged_cho.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere              0
type_local                   0
surface_reelle_bati          0
surface_terrain              0
nombre_pieces_principales    0
nombre_lots                  0
code_postal                  0
nom_commune                  0
code_commune                 0
longitude                    0
latitude                     0
year                         0
population                   0
rev_fisc_moyen_reference     0
unemployment_rate            0
dtype: int64


In [ ]:
df_merged_cho.shape[0]

129887

In [ ]:
df_merged_cho


,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,year,population,rev_fisc_moyen_reference,unemployment_rate
0,500000,Maison,127,638,4,0,34990,Juvignac,34123,3.800856,43.616081,2024,11344,28.72,28.32
1,144000,Appartement,47,0,2,2,34990,Juvignac,34123,3.809723,43.618576,2024,11344,28.72,28.32
2,183000,Appartement,68,0,3,2,34990,Juvignac,34123,3.811186,43.617937,2024,11344,28.72,28.32
3,255350,Appartement,78,0,3,2,34990,Juvignac,34123,3.803852,43.613413,2024,11344,28.72,28.32
4,174000,Appartement,59,0,3,1,34990,Juvignac,34123,3.801290,43.613778,2024,11344,28.72,28.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129882,355000,Maison,123,767,3,0,34800,Mérifons,34156,3.287057,43.622588,2021,45,25.85,0.00
129883,355000,Maison,123,1000,3,0,34800,Mérifons,34156,3.287057,43.622588,2021,45,25.85,0.00
129884,355000,Maison,123,500,3,0,34800,Mérifons,34156,3.287057,43.622588,2021,45,25.85,0.00
129885,236000,Maison,45,405,2,0,34800,Mérifons,34156,3.284967,43.620958,2020,46,25.85,0.00


Prix au m2 = valeur fonciere / surface reelle bati 

In [ ]:
df_merged_cho['prix_m2'] = round(df_merged_cho['valeur_fonciere'] / df_merged_cho['surface_reelle_bati'],1)

In [ ]:
missing_values_per_column = df_merged_cho.isnull().sum()
print("Missing values per column:\n", missing_values_per_column)

Missing values per column:
 valeur_fonciere              0
type_local                   0
surface_reelle_bati          0
surface_terrain              0
nombre_pieces_principales    0
nombre_lots                  0
code_postal                  0
nom_commune                  0
code_commune                 0
longitude                    0
latitude                     0
year                         0
population                   0
rev_fisc_moyen_reference     0
unemployment_rate            0
prix_m2                      0
dtype: int64


In [ ]:
df_merged_cho

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,year,population,rev_fisc_moyen_reference,unemployment_rate,prix_m2
0,500000,Maison,127,638,4,0,34990,Juvignac,34123,3.800856,43.616081,2024,11344,28.72,28.32,3937.0
1,144000,Appartement,47,0,2,2,34990,Juvignac,34123,3.809723,43.618576,2024,11344,28.72,28.32,3063.8
2,183000,Appartement,68,0,3,2,34990,Juvignac,34123,3.811186,43.617937,2024,11344,28.72,28.32,2691.2
3,255350,Appartement,78,0,3,2,34990,Juvignac,34123,3.803852,43.613413,2024,11344,28.72,28.32,3273.7
4,174000,Appartement,59,0,3,1,34990,Juvignac,34123,3.801290,43.613778,2024,11344,28.72,28.32,2949.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129882,355000,Maison,123,767,3,0,34800,Mérifons,34156,3.287057,43.622588,2021,45,25.85,0.00,2886.2
129883,355000,Maison,123,1000,3,0,34800,Mérifons,34156,3.287057,43.622588,2021,45,25.85,0.00,2886.2
129884,355000,Maison,123,500,3,0,34800,Mérifons,34156,3.287057,43.622588,2021,45,25.85,0.00,2886.2
129885,236000,Maison,45,405,2,0,34800,Mérifons,34156,3.284967,43.620958,2020,46,25.85,0.00,5244.4


In [ ]:
url_schools = "https://www.herault-data.fr/api/explore/v2.1/catalog/datasets/colleges_publics_herault/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"  
df_schools = pd.read_csv(url_schools, sep=';')

In [ ]:
df_schools.head(2)

,Nom Collège,Numéro UAI,Rue,Boite Postale,Code Postal,Commune,EPCI,Téléphone,Code insee,Site web,...,3eme Division,Total Effectif SEGPA,6eme effectif SEGPA,5eme effectif SEGPA,4eme effectif SEGPA,3eme effectif SEGPA,Total Division SEGPA,Section ULIS,geo_shape,geo_point_2d
0,Collège Alexandre Laissac,0340056B,place Alexandre Laissac,NaN,34390,OLARGUES,Communauté de communes du Minervois au Caroux,Tél : 04 67 97 72 08,34187,http://www.herault.fr/jeunesse/contact/college...,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0,"{""coordinates"": [2.9164102281811473, 43.556225...","43.55622539233105, 2.9164102281811473"
1,Collège Les Deux Pins,0340025T,rue Maurice Clavel,BP 309,34113,FRONTIGNAN,Communauté d'agglomération Sète Agglopôle Médi...,Tél : 04 67 18 69 50,34108,http://www.herault.fr/jeunesse/contact/college...,...,6.0,60.0,15.0,16.0,15.0,14.0,4.0,1,"{""coordinates"": [3.7511315567996744, 43.453554...","43.453554756037114, 3.7511315567996744"


In [ ]:
# Convertir les coordonnées en radians
def to_radians(df, lat_col, lon_col):
    return np.radians(df[[lat_col, lon_col]].values)

# Construire un KDTree pour accélérer la recherche
def calculate_distance_with_kdtree(df_merged_cho, df_schools, radius_km=1.0):  # Rayon défini à 1 km
    EARTH_RADIUS = 6371.0  # Rayon de la Terre (en kilomètres)

    # Convertir les coordonnées en radians
    immo_coords = to_radians(df_merged_cho, 'latitude', 'longitude')
    school_coords = to_radians(df_schools, 'geo_point_2d_lat', 'geo_point_2d_lon')

    # Construire le KDTree
    tree = KDTree(school_coords)

    counts = []
    min_distances = []

    for coord in immo_coords:
        # Rechercher les écoles dans le rayon spécifié
        indices = tree.query_ball_point(coord, r=radius_km / EARTH_RADIUS)  # Rayon défini à 1 km

        # Calculer la distance minimale (même s’il n’y a pas d’école dans le rayon)
        dists, _ = tree.query(coord, k=1)  # Rechercher l'école la plus proche
        min_distance_km = dists * EARTH_RADIUS  # Convertir la distance radian en kilomètres

        counts.append(len(indices))  # Compter les écoles dans un rayon de 1 km
        min_distances.append(round(min_distance_km, 2))  # Arrondir la distance minimale à deux décimales

    # Ajouter les résultats à df_merged_cho
    df_merged_cho['schools_within_1km'] = counts  # écoles dans un rayon de 1 km
    df_merged_cho['min_distance_km_school'] = min_distances  # distance minimale à une école

    return df_merged_cho

# Vérifier que geo_point_2d est correctement séparé en latitude et longitude
df_schools[['geo_point_2d_lat', 'geo_point_2d_lon']] = df_schools['geo_point_2d'].apply(lambda x: pd.Series(map(float, x.split(','))))

# Vérifier la cohérence des plages de coordonnées
print("Plage de latitude des propriétés :", df_merged_cho['latitude'].min(), df_merged_cho['latitude'].max())
print("Plage de latitude des écoles :", df_schools['geo_point_2d_lat'].min(), df_schools['geo_point_2d_lat'].max())

# Exécuter le calcul des distances
df_merged_cho = calculate_distance_with_kdtree(df_merged_cho, df_schools)

# Afficher les résultats
print(df_merged_cho[['latitude', 'longitude', 'schools_within_1km', 'min_distance_km_school']])


房產經緯度範圍： 43.229027 43.949466
學校經緯度範圍： 43.27095227028749 43.9364340924678
         latitude  longitude  schools_within_1km  min_distance_km_school
0       43.616081   3.800856                   0                    2.42
1       43.618576   3.809723                   0                    1.44
2       43.617937   3.811186                   0                    1.27
3       43.613413   3.803852                   0                    2.12
4       43.613778   3.801290                   0                    2.40
...           ...        ...                 ...                     ...
129882  43.622588   3.287057                   0                   12.35
129883  43.622588   3.287057                   0                   12.35
129884  43.622588   3.287057                   0                   12.35
129885  43.620958   3.284967                   0                   12.58
129886  43.620958   3.284967                   0                   12.58

[129887 rows x 4 columns]


In [ ]:
df_merged_cho

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,year,population,rev_fisc_moyen_reference,unemployment_rate,prix_m2,schools_within_1km,min_distance_km_school
0,500000,Maison,127,638,4,0,34990,Juvignac,34123,3.800856,43.616081,2024,11344,28.72,28.32,3937.0,0,2.42
1,144000,Appartement,47,0,2,2,34990,Juvignac,34123,3.809723,43.618576,2024,11344,28.72,28.32,3063.8,0,1.44
2,183000,Appartement,68,0,3,2,34990,Juvignac,34123,3.811186,43.617937,2024,11344,28.72,28.32,2691.2,0,1.27
3,255350,Appartement,78,0,3,2,34990,Juvignac,34123,3.803852,43.613413,2024,11344,28.72,28.32,3273.7,0,2.12
4,174000,Appartement,59,0,3,1,34990,Juvignac,34123,3.801290,43.613778,2024,11344,28.72,28.32,2949.2,0,2.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129882,355000,Maison,123,767,3,0,34800,Mérifons,34156,3.287057,43.622588,2021,45,25.85,0.00,2886.2,0,12.35
129883,355000,Maison,123,1000,3,0,34800,Mérifons,34156,3.287057,43.622588,2021,45,25.85,0.00,2886.2,0,12.35
129884,355000,Maison,123,500,3,0,34800,Mérifons,34156,3.287057,43.622588,2021,45,25.85,0.00,2886.2,0,12.35
129885,236000,Maison,45,405,2,0,34800,Mérifons,34156,3.284967,43.620958,2020,46,25.85,0.00,5244.4,0,12.58


In [ ]:
df_bus = pd.read_csv('gtfs-herault-stops.csv',delimiter=';')    

In [ ]:
df_bus.head(2)

,﻿ID,Name,commune,epci,Coordinates,Wheelchair Boarding,Nom département,Code département
0,1004732.0,POUSSAN - Coopérative,Poussan,Communauté d'agglomération Sète Agglopôle Médi...,"43.484235, 3.671101",0,Hérault,34
1,1004494.0,POUZOLS - Rte des Lauzes,Pouzols,Communauté de communes Vallée de l'Hérault,"43.616558, 3.512229",0,Hérault,34


In [ ]:

# Convertir les coordonnées en radians
def to_radians(df, lat_col, lon_col):
    return np.radians(df[[lat_col, lon_col]].values)

# Construire un KDTree pour accélérer la recherche
def calculate_distance_with_kdtree(df_merged_cho, df_bus, radius_km=0.5):  # Rayon défini à 500 mètres (0,5 km)
    EARTH_RADIUS = 6371.0  # Rayon de la Terre (en kilomètres)

    # Convertir les coordonnées en radians
    immo_coords = to_radians(df_merged_cho, 'latitude', 'longitude')
    bus_coords = to_radians(df_bus, 'bus_latitude', 'bus_longitude')

    # Construire le KDTree
    tree = KDTree(bus_coords)

    counts = []
    min_distances = []

    for coord in immo_coords:
        # Rechercher les arrêts de bus dans le rayon spécifié
        indices = tree.query_ball_point(coord, r=radius_km / EARTH_RADIUS)  # Rayon défini à 500 mètres

        # Calculer la distance minimale (même s’il n’y a pas d’arrêt de bus dans le rayon)
        dists, _ = tree.query(coord, k=1)  # Rechercher l'arrêt de bus le plus proche
        min_distance_km = dists * EARTH_RADIUS  # Convertir la distance en kilomètres

        counts.append(len(indices))  # Compter les arrêts de bus dans un rayon de 500 mètres
        min_distances.append(round(min_distance_km, 2))  # Arrondir la distance minimale à deux décimales

    # Ajouter les résultats à df_merged_cho
    df_merged_cho['bus_500m'] = counts
    df_merged_cho['min_km_bus'] = min_distances  # Nommer les résultats

    return df_merged_cho

# Vérifier que les coordonnées sont correctement séparées en latitude et longitude
df_bus[['bus_latitude', 'bus_longitude']] = df_bus['Coordinates'].apply(lambda x: pd.Series(map(float, x.split(','))))

# Vérifier les plages de coordonnées
print("Plage de latitude des propriétés :", df_merged_cho['latitude'].min(), df_merged_cho['latitude'].max())
print("Plage de latitude des arrêts de bus :", df_bus['bus_latitude'].min(), df_bus['bus_latitude'].max())

# Exécuter le calcul de distance
df_merged_cho = calculate_distance_with_kdtree(df_merged_cho, df_bus)


房產經緯度範圍： 43.229027 43.949466
巴士站經緯度範圍： 43.248829 43.94556
         latitude  longitude  bus_500m  min_km_bus
0       43.616081   3.800856         0        2.03
1       43.618576   3.809723         0        1.04
2       43.617937   3.811186         0        0.87
3       43.613413   3.803852         0        1.74
4       43.613778   3.801290         0        2.01
...           ...        ...       ...         ...
129882  43.622588   3.287057         0        1.29
129883  43.622588   3.287057         0        1.29
129884  43.622588   3.287057         0        1.29
129885  43.620958   3.284967         0        1.11
129886  43.620958   3.284967         0        1.11

[129887 rows x 4 columns]


In [ ]:
df_merged_cho[df_merged_cho['bus_500m'] > 0]

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,latitude,year,population,rev_fisc_moyen_reference,unemployment_rate,prix_m2,schools_within_1km,min_distance_km_school,bus_500m,min_km_bus
296,377000,Maison,103,422,5,0,34990,Juvignac,34123,3.816766,43.613677,2022,11084,28.72,28.32,3660.2,1,0.75,6,0.40
694,383400,Maison,98,546,4,0,34990,Juvignac,34123,3.816815,43.613085,2021,10864,28.72,28.32,3912.2,1,0.78,2,0.44
1019,47000,Appartement,16,0,1,1,34000,Montpellier,34172,3.881803,43.601744,2024,295542,22.41,54.75,2937.5,1,0.62,2,0.36
1025,410000,Appartement,37,0,1,1,34000,Montpellier,34172,3.878425,43.605253,2024,295542,22.41,54.75,11081.1,1,0.73,2,0.19
1026,410000,Appartement,56,0,2,1,34000,Montpellier,34172,3.878425,43.605253,2024,295542,22.41,54.75,7321.4,1,0.73,2,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129876,435000,Maison,114,620,4,0,34160,Garrigues,34112,4.018293,43.794577,2022,177,28.66,22.33,3815.8,0,12.78,3,0.47
129877,200000,Maison,114,99,4,0,34160,Garrigues,34112,4.013894,43.794558,2021,174,28.66,22.33,1754.4,0,12.66,3,0.05
129878,200000,Appartement,120,59,5,0,34160,Garrigues,34112,4.014560,43.794158,2020,174,28.66,22.33,1666.7,0,12.63,3,0.12
129879,200000,Appartement,65,59,3,0,34160,Garrigues,34112,4.014560,43.794158,2020,174,28.66,22.33,3076.9,0,12.63,3,0.12


In [ ]:
df_merged_cho.to_csv("projet_immo.csv", index=False)

In [ ]:
df_merged_cho.columns

Index(['valeur_fonciere', 'type_local', 'surface_reelle_bati',
       'surface_terrain', 'nombre_pieces_principales', 'nombre_lots',
       'code_postal', 'nom_commune', 'code_commune', 'longitude', 'latitude',
       'year', 'population', 'rev_fisc_moyen_reference', 'unemployment_rate',
       'prix_m2', 'schools_within_1km', 'min_distance_km_school', 'bus_500m',
       'min_km_bus'],
      dtype='object')

In [ ]:
# calculer taux de croissance pour 2019 - 2024 

df_merged_cho["popu_croissance"] = round(df_merged_cho["population"].pct_change(),2)

# taux de croissance pour 2019（utilise 2020）
df_merged_cho.loc[0, "popu_croissance"] = df_merged_cho.loc[1, "popu_croissance"]

# en %
df_merged_cho["popu_croissance"] = df_merged_cho["popu_croissance"] * 100

In [ ]:
df_merged_cho

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,...,year,population,rev_fisc_moyen_reference,unemployment_rate,prix_m2,schools_within_1km,min_distance_km_school,bus_500m,min_km_bus,popu_croissance
0,500000,Maison,127,638,4,0,34990,Juvignac,34123,3.800856,...,2024,11344,28.72,28.32,3937.0,0,2.42,0,2.03,0.0
1,144000,Appartement,47,0,2,2,34990,Juvignac,34123,3.809723,...,2024,11344,28.72,28.32,3063.8,0,1.44,0,1.04,0.0
2,183000,Appartement,68,0,3,2,34990,Juvignac,34123,3.811186,...,2024,11344,28.72,28.32,2691.2,0,1.27,0,0.87,0.0
3,255350,Appartement,78,0,3,2,34990,Juvignac,34123,3.803852,...,2024,11344,28.72,28.32,3273.7,0,2.12,0,1.74,0.0
4,174000,Appartement,59,0,3,1,34990,Juvignac,34123,3.801290,...,2024,11344,28.72,28.32,2949.2,0,2.40,0,2.01,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129882,355000,Maison,123,767,3,0,34800,Mérifons,34156,3.287057,...,2021,45,25.85,0.00,2886.2,0,12.35,0,1.29,29.0
129883,355000,Maison,123,1000,3,0,34800,Mérifons,34156,3.287057,...,2021,45,25.85,0.00,2886.2,0,12.35,0,1.29,0.0
129884,355000,Maison,123,500,3,0,34800,Mérifons,34156,3.287057,...,2021,45,25.85,0.00,2886.2,0,12.35,0,1.29,0.0
129885,236000,Maison,45,405,2,0,34800,Mérifons,34156,3.284967,...,2020,46,25.85,0.00,5244.4,0,12.58,0,1.11,2.0


In [ ]:
df_merged_cho.to_csv("projet_immo.csv", index=False)

"projet_immo.csv" contient les données nettoyées jusqu'à présent. 
Nous allons ajouter les valeurs potentiellement utils pour optimiser le modèle de la prédiction.  

*Part 2 : Ajouter le Taux d'intérêt 
Taux d'interet de la BCE (banque centrale europenne): https://fr.statista.com/statistiques/571395/fluctuation-des-taux-d-interet-de-la-bce/

In [6]:
df_interet=pd.read_excel("taux_interet.xlsx")
df_interet.head(10)

,date,taux d'interet
0,2008-10-15,3.75
1,2008-11-12,3.25
2,2008-12-10,2.50
3,2009-01-21,2.00
4,2009-03-11,1.50
5,2009-04-08,1.25
6,2009-05-13,1.00
7,2011-04-13,1.25
8,2011-07-13,1.50
9,2011-11-09,1.25


Pour la granularité, on va fitrer les données en calculant la moyenne par année avec Groupby

In [8]:
df_interet['date'] = pd.to_datetime(df_interet['date'], format='%Y-%m-%d', errors='coerce')
df_interet['year'] = df_interet['date'].dt.year
df_interet.head(2)

,date,taux d'interet,year
0,2008-10-15,3.75,2008
1,2008-11-12,3.25,2008


In [13]:
moyenne_par_annee = df_interet.groupby('year')["taux d'interet"].mean().reset_index()
moyenne_par_annee

,year,taux d'interet
0,2008,3.166667
1,2009,1.437500
2,2011,1.250000
3,2012,0.750000
4,2013,0.375000
5,2014,0.100000
6,2015,0.050000
7,2016,0.000000
8,2019,0.000000
9,2022,1.562500


Étant donné l’absence de données pour les années 2020 et 2021, sans doute à cause de la crise du COVID-19, j’ai complété les années manquantes entre 2008 et 2025 en recopiant la valeur de l’année précédente.

In [15]:
all_years = pd.DataFrame({'year': list(range(2008, 2026))})
df_interet = all_years.merge(moyenne_par_annee, on='year', how='left')
df_interet["taux d'interet"] = df_interet["taux d'interet"].fillna(method='ffill')
df_interet.head(2)

,year,taux d'interet
0,2008,3.166667
1,2009,1.437500


Dans le cadre de la préparation des données pour l'entraînement du modèle, le taux d'intérêt est conservé sous forme numérique (par exemple : 3.75 pour 3.75 %). Il n’est pas nécessaire de le convertir en pourcentage affiché (avec le symbole %) car les modèles de machine learning nécessitent des valeurs numériques pour fonctionner correctement.

Le format actuel est donc adapté pour les traitements algorithmiques, tout en représentant déjà un taux en pourcentage implicite. Aucune transformation supplémentaire n’est requise à ce stade.

** Jointure 
Taux d'intérêt + projet_immo

In [17]:
projet_immo = pd.read_csv("projet_immo.csv")
df_joint = projet_immo.merge(df_interet, on='year', how='left')
df_joint

,valeur_fonciere,type_local,surface_reelle_bati,surface_terrain,nombre_pieces_principales,nombre_lots,code_postal,nom_commune,code_commune,longitude,...,population,rev_fisc_moyen_reference,unemployment_rate,prix_m2,schools_within_1km,min_distance_km_school,bus_500m,min_km_bus,popu_croissance,taux d'interet
0,500000,Maison,127,638,4,0,34990,Juvignac,34123,3.800856,...,11344,28.72,28.32,3937.0,0,2.42,0,2.03,0.0,3.6125
1,144000,Appartement,47,0,2,2,34990,Juvignac,34123,3.809723,...,11344,28.72,28.32,3063.8,0,1.44,0,1.04,0.0,3.6125
2,183000,Appartement,68,0,3,2,34990,Juvignac,34123,3.811186,...,11344,28.72,28.32,2691.2,0,1.27,0,0.87,0.0,3.6125
3,255350,Appartement,78,0,3,2,34990,Juvignac,34123,3.803852,...,11344,28.72,28.32,3273.7,0,2.12,0,1.74,0.0,3.6125
4,174000,Appartement,59,0,3,1,34990,Juvignac,34123,3.801290,...,11344,28.72,28.32,2949.2,0,2.40,0,2.01,0.0,3.6125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129882,355000,Maison,123,767,3,0,34800,Mérifons,34156,3.287057,...,45,25.85,0.00,2886.2,0,12.35,0,1.29,29.0,0.0000
129883,355000,Maison,123,1000,3,0,34800,Mérifons,34156,3.287057,...,45,25.85,0.00,2886.2,0,12.35,0,1.29,0.0,0.0000
129884,355000,Maison,123,500,3,0,34800,Mérifons,34156,3.287057,...,45,25.85,0.00,2886.2,0,12.35,0,1.29,0.0,0.0000
129885,236000,Maison,45,405,2,0,34800,Mérifons,34156,3.284967,...,46,25.85,0.00,5244.4,0,12.58,0,1.11,2.0,0.0000


** Part de Modèle

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# creat LabelEncoder
le_code_postal = LabelEncoder()
le_nom_commune = LabelEncoder()

# str - int
df_merged_cho["code_postal_encoded"] = le_code_postal.fit_transform(df_merged_cho["code_postal"])
df_merged_cho["code_commune_encoded"] = le_nom_commune.fit_transform(df_merged_cho["code_commune"])

# modèle
surface_reelle_bati, nombre_pieces_principales, prix_m2
code_postal_encoded
population, rev_fisc_moyen_reference, unemployment_rate
schools_within_1km, min_distance_km_school, bus_500m, min_km_bus

In [ ]:
columns_to_keep = ['valeur_fonciere','surface_reelle_bati', 'nombre_pieces_principales', 'prix_m2', 
                   'code_postal_encoded', 'popu_croissance', 'rev_fisc_moyen_reference', 
                   'unemployment_rate', 'schools_within_1km', 'min_distance_km_school', 
                   'bus_500m', 'min_km_bus']

# charger les donnée selectionnées dans  df_immo
df_immo = df_merged_cho[columns_to_keep]

In [ ]:
from sklearn.model_selection import train_test_split

# definir X et y
X = df_immo.drop(columns=["valeur_fonciere"])  # drop target variable
y = df_immo["valeur_fonciere"]

# traning set（80%）& testing set（20%）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"training set: {X_train.shape}")
print(f"testing set: {X_test.shape}")


training set: (103909, 11)
testing set: (25978, 11)


Random Forest model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Initialize the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict house prices
y_pred = model.predict(X_test)

# Evaluate model performance
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Model evaluation results:")
print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")
print(f"R² (Coefficient of Determination): {r2:.2f}")


Model evaluation results:
RMSE (Root Mean Squared Error): 55628.36
R² (Coefficient of Determination): 1.00


dataset pour test

In [ ]:
import pandas as pd
import numpy as np

# Exemple de données pour tester le modèle
data_test = {
    'surface_reelle_bati': [120, 85, 150, 200, 95],  # Surface en m²
    'nombre_pieces_principales': [5, 3, 6, 7, 4],    # Nombre de pièces principales
    'prix_m2': [3500, 4000, 3300, 3600, 4200],       # Prix par m²
    'code_postal_encoded': [3, 7, 1, 4, 5],          # Code postal encodé
    'popu_croissance': [0.02, 0.01, 0.03, 0.02, 0.01],  # Croissance de la population
    'rev_fisc_moyen_reference': [25000, 35000, 30000, 32000, 27000],  # Revenu fiscal moyen
    'unemployment_rate': [0.05, 0.08, 0.04, 0.06, 0.07],  # Taux de chômage
    'schools_within_1km': [1, 2, 3, 2, 1],          # Nombre d'écoles dans un rayon de 1 km
    'min_distance_km_school': [0.5, 0.7, 0.3, 0.8, 0.6],  # Distance minimale à l'école en km
    'bus_500m': [1, 1, 0, 1, 1],                    # Présence de bus dans un rayon de 500 m
    'min_km_bus': [0.2, 0.3, 0.5, 0.1, 0.2]         # Distance minimale à l'arrêt de bus en km
}

# Création du DataFrame
df_test = pd.DataFrame(data_test)

# Vérification des données
print(df_test.head())

# Prédiction sur ces données
y_test_pred = model.predict(df_test)

# Affichage des résultats
print("Prédictions de prix des maisons :")
print(y_test_pred)


   surface_reelle_bati  nombre_pieces_principales  prix_m2  \
0                  120                          5     3500   
1                   85                          3     4000   
2                  150                          6     3300   
3                  200                          7     3600   
4                   95                          4     4200   

   code_postal_encoded  popu_croissance  rev_fisc_moyen_reference  \
0                    3             0.02                     25000   
1                    7             0.01                     35000   
2                    1             0.03                     30000   
3                    4             0.02                     32000   
4                    5             0.01                     27000   

   unemployment_rate  schools_within_1km  min_distance_km_school  bus_500m  \
0               0.05                   1                     0.5         1   
1               0.08                   2                

In [ ]:
import joblib

# concerver le modèle
joblib.dump(model, 'house_price_predictor.pkl')

['house_price_predictor.pkl']

In [ ]:
pip install anyio==3.5.0


  Using cached anyio-3.5.0-py3-none-any.whl (79 kB)
  Attempting uninstall: anyio
    Found existing installation: anyio 4.8.0
    Uninstalling anyio-4.8.0:
      Successfully uninstalled anyio-4.8.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
starlette 0.45.3 requires anyio<5,>=3.6.2, but you have anyio 3.5.0 which is incompatible.


In [ ]:
pip install fastapi uvicorn scikit-learn pandas

  Using cached anyio-4.8.0-py3-none-any.whl (96 kB)
  Attempting uninstall: anyio
    Found existing installation: anyio 3.5.0
    Uninstalling anyio-3.5.0:
      Successfully uninstalled anyio-3.5.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.8.0 which is incompatible.


In [ ]:
pip list

Package                       Version
----------------------------- ---------------
alabaster                     0.7.12
anaconda-client               1.11.2
anaconda-navigator            2.4.0
annotated-types               0.7.0
anyio                         4.8.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
atomicwrites                  1.4.0
attrs                         22.1.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.11.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
bcrypt                        3.2.0
beautifulsoup4                4.12.2
binaryornot                   0.4.4
black                         0.0
bleach   

In [ ]:
pip install --upgrade typing-extensions


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade fastapi


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade typing-extensions


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np

In [ ]:


# Charger le modèle
with open('house_price_predictor.pkl', 'rb') as f:
    model = joblib.load(f)

# Initialisation de FastAPI
app = FastAPI()

# Schéma des données d'entrée
class PredictionRequest(BaseModel):
    surface_reelle_bati: float
    nombre_pieces_principales: float
    prix_m2: float
    code_postal_encoded: float
    popu_croissance: float
    rev_fisc_moyen_reference: float
    unemployment_rate: float
    schools_within_1km: float
    min_distance_km_school: float
    bus_500m: float
    min_km_bus: float

@app.post("/predict")
def predict(data: PredictionRequest):
    # Préparer les données d'entrée en tant que tableau numpy
    features = np.array([[data.surface_reelle_bati, data.nombre_pieces_principales, data.prix_m2,
                          data.code_postal_encoded, data.popu_croissance, data.rev_fisc_moyen_reference,
                          data.unemployment_rate, data.schools_within_1km, data.min_distance_km_school,
                          data.bus_500m, data.min_km_bus]])
    
    # Effectuer la prédiction avec le modèle
    prediction = model.predict(features)
    
    # Retourner la prédiction sous forme de liste
    return {"prediction": prediction.tolist()}

# Lancer l'application FastAPI avec Uvicorn
# Exécution dans le terminal :
# uvicorn your_filename:app --reload


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import joblib

In [ ]:
#Charger le modèle
model = joblib.load('house_price_predictor.pkl')

#Initialiser l'application FastAPI
app = FastAPI(title="API de Prédiction de Prix des Maisons")

#Définir le schéma des données attendues
class HouseData(BaseModel):
    surface_reelle_bati: float
    nombre_pieces_principales: int
    prix_m2: float
    code_postal_encoded: int
    popu_croissance: float
    rev_fisc_moyen_reference: float
    unemployment_rate: float
    schools_within_1km: int
    min_distance_km_school: float
    bus_500m: int
    min_km_bus: float

#Route de test pour vérifier que l'API fonctionne
@app.get("/")
def read_root():
    return {"message": "Bienvenue sur l'API de Prédiction de Prix des Maisons !"}

#Endpoint pour effectuer des prédictions
@app.post("/predict")
def predict_price(data: HouseData):
    # Conversion des données en DataFrame
    df = pd.DataFrame([data.dict()])

    # Prédiction
    prediction = model.predict(df)

    # Retourner le résultat
    return {"prediction": prediction[0]}


In [ ]:
pip install fastapi uvicorn
uvicorn app:app --reload

SyntaxError: invalid syntax (260581419.py, line 1)